# Import libraries

In [4]:
import numpy as np
import pandas as pd
import geopandas as gpd
import requests

# Set feature service variables

In [5]:
feat_srvc = 'https://services3.arcgis.com/0Fs3HcaFfvzXvm7w/ArcGIS/rest/services/CRIS_Zonal_Statistics_by_County/FeatureServer/3'
query = '/query'

# Use Geographic Identifiers (Field Name: GEOID) to pick which counties to calculate zonal statistics over.
uniqueID = 'GEOID'

# Pick counties and years to return

In [6]:
# Put each county's GEOID between apostrophes and separate by commas, e.g. ids = '20201', '20117'
# Set ids to 'all' if using all counties in the feature layer
ids = '20201', '20203', '20117'

In [7]:
# Pick start and end year of the period you want to process.
# If you only want to process 1 year, put your desired year for both the start and end.
year_start = 1981
year_end = 2020

In [8]:
# Define the where clause for counties and years to return

if ids == 'all':
    
    where_clause_id = f"YEAR >= {year_start} AND YEAR <= {year_end}"

else:
    
    if np.size(ids) == 1:

        where_clause_id = f"{uniqueID} = '{ids}' AND YEAR >= {year_start} AND YEAR <= {year_end}"

    else:

        where_clause_id = f"{uniqueID} IN {ids} AND YEAR >= {year_start} AND YEAR <= {year_end}"

# Pick variables to return

### Pick one of the three options below, run the appropriate cells, then move to "Retrieve data"

##### Option 1: return all variables

In [64]:
# Run this cell if you want all variables returned
variables = '*'

##### Option 2: return manually selected subset of variables

In [ ]:
# Show available variables
response_temp = requests.get(feat_srvc + '?f=pjson')
data = response_temp.json()
fields = data['fields']
for field in fields:
    print('Variable:', field['name'], '\nDescription:', field['alias'], '\n')
    print()

In [41]:
# If you want 1 or more variables from the above list, select them here
# Put each variable between apostrophes and separate by commas, e.g. variables = 'TMAX_NCLIMGRID_MIN'
variables = 'TMAX_NCLIMGRID_MIN', 'TMAX_NCLIMGRID_MAX', 'TMAX_NCLIMGRID_MEAN'

##### Option 3: select all NCLIMGRID or all LIVNEH variables

In [35]:
# Run the line below for all NCLIMGRID variables
variables = [field['name'] for field in fields if "NCLIMGRID" in field['name']]

# Run the line below for all NCLIMGRID variables
# variables = [field['name'] for field in fields if "LIVNEH" in field['name']]

# Retrieve data

In [65]:
# Add counties ('GEOID') and 'YEAR' to the variables-to-return list
return_variables = ['GEOID', 'YEAR']

if np.size(variables) == 1:

    return_variables.append(variables)

else:

    for var in variables:

        return_variables.append(var)

In [88]:
# Maximum number of variables that can be processed in each request: 50
# Maximum number of rows that can be processed in each request: 1000
# This code loops through variables and rows and appends each subset (1000x50) to a final GeoDataFrame (gdf)

# Number of variables that are requested to be processed
vars_to_process = len(return_variables)

# Number of variables to process in each loop
loop_size = 50

# Set initial value for loop number (v)
v = 0

# Loop through variables, 50 at one time
# While there are variables left to process, keep looping through this code
while vars_to_process > 0:

    # Sequentially subset the variables to process with 50 variables in each loop
    return_variables_condition = return_variables[max((loop_size*v - 1) + 1, 0): min(loop_size*(v+1), len(return_variables))]

    # Update number of variables to process, i.e. subtract 50 (loop_size)
    vars_to_process -= loop_size
    
    # Update loop number (v)
    v += 1

    # Set parameters for requests.get()
    params = {
        'where': where_clause_id,
        'outFields': return_variables_condition,
        'orderByFields': ['YEAR', 'GEOID'],
        'f': 'pgeojson',
    }

    # Request response from feature service
    response = requests.get(feat_srvc + query, params=params)


    # Put the first 50 variables in gdf to establish the GeoDataFrame
    if v == 1:
        
        # Translate the reponse.text into a temporary GeoDataFrame
        gdf_var1_temp = gpd.read_file(response.text)

        # Drop geometry field
        gdf_var1_temp.drop(columns=['geometry'], inplace=True)

        # Establish the final GeoDataFrame (gdf) and put the first subset of rows and variables (temporary GeoDataFrame) in it
        gdf = gdf_var1_temp


        # Set initial value for loop number (i)
        i = 0

        # Loop through rows, 1000 at one time
        while len(gdf_var1_temp) == 1000:
            
            # Update loop number (i)
            i = i + 1

            # Set the offset for in params. This says how many rows to skip from the start.
            offset = 1000 * i

            # Set parameters for requests.get()
            params = {
                'where': where_clause_id,
                'outFields': return_variables_condition,
                'orderByFields': ['YEAR', 'GEOID'],
                'f': 'pgeojson',
                'resultOffset': f'{offset}'
            }

            # Request response from feature service
            response = requests.get(feat_srvc + query, params=params)

            # Translate the reponse.text into a temporary GeoDataFrame
            gdf_var1_temp = gpd.read_file(response.text)

            # Concatenate the new subset to the final GeoDataFrame (gdf)
            gdf = pd.concat([gdf, gdf_var1_temp], ignore_index=True);

            # Drop geometry field
            gdf.drop(columns=['geometry'], inplace=True)


    # Put all the additional variables, beyond the first 50, in a temporary GeoDataFrame and concatenate with the final GeoDataFrame (gdf)
    elif v > 1:
    
        # Translate the reponse.text into a temporary GeoDataFrame
        gdf_var2_temp = gpd.read_file(response.text)
        
        # Drop geometry field
        gdf_var2_temp.drop(columns=['geometry'], inplace=True)

        # Establish the a temporart GeoDataFrame (gdf_var2) and put the first subset of rows and variables (temporary GeoDataFrame) in it     
        gdf_var2 = gdf_var2_temp


        # Set initial value for loop number (j)
        j = 0

        # Loop through rows, 1000 at one time
        while len(gdf_var2_temp) == 1000:

            # Update loop number (j)
            j = j + 1

            # Set the offset for in params. This says how many rows to skip from the start.
            offset = 1000 * j

            # Set parameters for requests.get()
            params = {
                'where': where_clause_id,
                'outFields': return_variables_condition,
                'orderByFields': ['YEAR', 'GEOID'],
                'f': 'pgeojson',
                'resultOffset': f'{offset}'
            }

            # Request response from feature service
            response = requests.get(feat_srvc + query, params=params)

            # Translate the reponse.text into a temporary GeoDataFrame
            gdf_var2_temp = gpd.read_file(response.text)

            # Concatenate the new subset to the temporary GeoDataFrame (gdf_var2)
            gdf_var2 = pd.concat([gdf_var2, gdf_var2_temp], ignore_index=True)

            # Drop geometry field
            gdf_var2.drop(columns=['geometry'], inplace=True)

        # Concatenate temporary GeoDataFrame (gdf_var2) to the final GeoDataFrame (gdf)
        gdf = pd.concat([gdf, gdf_var2], axis=1)

In [89]:
gdf

,OBJECTID,GEOID,YEAR,BATCH_ID,TMAX_NCLIMGRID_MIN,TMAX_NCLIMGRID_MEAN,TMAX_NCLIMGRID_MAX,TMAX_LIVNEH_MIN,TMAX_LIVNEH_MEAN,TMAX_LIVNEH_MAX,...,PRDAYSABVNZ95TH_NCLIMGRID_MAX,PRDAYSABVNZ95TH_LIVNEH_MIN,PRDAYSABVNZ95TH_LIVNEH_MEAN,PRDAYSABVNZ95TH_LIVNEH_MAX,PRDAYSABVNZ99TH_NCLIMGRID_MIN,PRDAYSABVNZ99TH_NCLIMGRID_MEAN,PRDAYSABVNZ99TH_NCLIMGRID_MAX,PRDAYSABVNZ99TH_LIVNEH_MIN,PRDAYSABVNZ99TH_LIVNEH_MEAN,PRDAYSABVNZ99TH_LIVNEH_MAX
0,97657,20203,1981,1,67.53097,68.30665,69.10828,69.47655,69.94707,70.32584,...,6,3.0,5.897959,10.0,0,0.000000,0,0.0,0.510204,2.0
1,97936,20201,1981,1,64.78632,65.77835,66.67986,65.21297,67.39359,68.64093,...,10,5.0,9.571428,13.0,0,0.007143,1,0.0,0.984127,3.0
2,97940,20117,1981,1,64.24828,65.39116,66.42273,65.68484,66.49209,67.49890,...,10,5.0,9.079365,14.0,0,0.123077,2,0.0,1.476190,4.0
3,100292,20201,1982,1,60.78070,62.19161,63.25523,61.01163,63.62457,64.83636,...,6,2.0,5.126984,9.0,0,0.035714,1,0.0,0.428571,1.0
4,100293,20117,1982,1,60.56945,61.99343,63.10829,62.06286,63.06009,63.98860,...,11,1.0,7.793651,13.0,0,0.561539,2,0.0,1.682540,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,215239,20117,2019,1,61.83307,63.17385,64.10280,NaN,NaN,NaN,...,12,NaN,NaN,NaN,0,1.061538,2,NaN,NaN,NaN
116,216448,20203,2019,1,65.76391,66.39247,67.20715,NaN,NaN,NaN,...,2,NaN,NaN,NaN,0,0.000000,0,NaN,NaN,NaN
117,218909,20201,2020,1,64.10688,65.41978,66.43526,NaN,NaN,NaN,...,6,NaN,NaN,NaN,0,0.871429,1,NaN,NaN,NaN
118,218911,20117,2020,1,64.03914,65.41587,66.47530,NaN,NaN,NaN,...,6,NaN,NaN,NaN,0,0.100000,1,NaN,NaN,NaN


# Analysis examples

### Calculation 1: mean over time period

In [90]:
# If desired, run this cell to take a subset of the time period over which to calculate the mean
year_start_subset = 1990
year_end_subset = 1995

gdf = gdf[(gdf['YEAR'] >= year_start_subset) & (gdf['YEAR'] < year_end_subset+1)]

##### Run this cell to return the mean of each variable in the GeoDataFrame

In [91]:
# Group by county (GEOID) and calculate mean of period 'year_start_subset - year_end_subset'
variables_mean = gdf.groupby('GEOID').mean()

# Drop YEAR column
variables_mean.drop(columns='YEAR', inplace=True)

# If returning all variables, remove OBJECTID and BATCH_ID columns
if variables == '*':
    variables_mean.drop(columns=['OBJECTID', 'BATCH_ID'], inplace=True)

# Add index numbers to rows
variables_mean.reset_index(inplace=True)

# Rename column names to reflect that they represent the mean of each variable over time
variables_mean.rename(columns={gdf.columns[i]: f"{gdf.columns[i]} (mean)" for i in range(2, len(gdf.columns))}, inplace=True)

# Show means of variables
variables_mean

,GEOID,TMAX_NCLIMGRID_MIN (mean),TMAX_NCLIMGRID_MEAN (mean),TMAX_NCLIMGRID_MAX (mean),TMAX_LIVNEH_MIN (mean),TMAX_LIVNEH_MEAN (mean),TMAX_LIVNEH_MAX (mean),TAVG_NCLIMGRID_MIN (mean),TAVG_NCLIMGRID_MEAN (mean),TAVG_NCLIMGRID_MAX (mean),...,PRDAYSABVNZ95TH_NCLIMGRID_MAX (mean),PRDAYSABVNZ95TH_LIVNEH_MIN (mean),PRDAYSABVNZ95TH_LIVNEH_MEAN (mean),PRDAYSABVNZ95TH_LIVNEH_MAX (mean),PRDAYSABVNZ99TH_NCLIMGRID_MIN (mean),PRDAYSABVNZ99TH_NCLIMGRID_MEAN (mean),PRDAYSABVNZ99TH_NCLIMGRID_MAX (mean),PRDAYSABVNZ99TH_LIVNEH_MIN (mean),PRDAYSABVNZ99TH_LIVNEH_MEAN (mean),PRDAYSABVNZ99TH_LIVNEH_MAX (mean)
0,20117,62.525172,63.742053,64.834242,63.017218,63.891928,64.831152,51.788208,52.520252,53.131945,...,7.000000,4.833333,7.851851,10.833333,0.166667,0.960257,2.166667,0.333333,2.007936,3.666667
1,20201,63.295928,64.146538,64.979055,63.475293,65.151422,66.179448,52.213440,52.843330,53.427900,...,6.666667,4.500000,6.992063,9.833333,0.333333,1.017857,2.000000,0.166667,1.690476,3.500000
2,20203,65.892865,66.478455,66.977767,65.610580,66.098677,66.689188,52.181175,52.606673,53.136680,...,4.500000,2.666667,4.646258,7.666667,0.000000,0.311295,0.500000,0.000000,0.397959,1.000000


### Calculation 2: change over time

In [72]:
# Set years over which to calculate the change
year_start_change = 1985
year_end_change = 1995

# Create two new GeoDataFrames which contain only the the start and end years, respectively
gdf_start_change = gdf[(gdf['YEAR'] == year_start_change)]
gdf_end_change = gdf[(gdf['YEAR'] == year_end_change)]

In [73]:
# Set the indices of both GeoDataFrames to the counties (GEOIDs)
gdf_start_change.set_index('GEOID', inplace=True);
gdf_end_change.set_index('GEOID', inplace=True);

# Drop YEAR, MODEL_SET, and MODEL columns
gdf_start_change.drop(columns=['YEAR'], inplace=True)
gdf_end_change.drop(columns=['YEAR'], inplace=True)

# If returning all variables, remove OBJECTID and BATCH_ID columns
if variables == '*':
    gdf_start_change.drop(columns=['OBJECTID', 'BATCH_ID'], inplace=True)
    gdf_end_change.drop(columns=['OBJECTID', 'BATCH_ID'], inplace=True)

C:\Users\raf14049\AppData\Local\Temp\1\ipykernel_31584\4286477285.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf_start_change.drop(columns=['YEAR'], inplace=True)
C:\Users\raf14049\AppData\Local\Temp\1\ipykernel_31584\4286477285.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf_end_change.drop(columns=['YEAR'], inplace=True)
C:\Users\raf14049\AppData\Local\Temp\1\ipykernel_31584\4286477285.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

In [74]:
# Calculate absolute difference between start and end years
variables_difference_absolute = gdf_end_change - gdf_start_change

# Add index numbers to rows
variables_difference_absolute.reset_index(inplace=True)

# Rename columns to reflect that they represent the mean of each variable over time
variables_difference_absolute.rename(columns={gdf.columns[i]: f"{gdf.columns[i]} (change)" for i in range(2, len(gdf.columns))}, inplace=True)

# Show absolute difference of each county and each variable between the start and end years
variables_difference_absolute

,GEOID,TMAX_NCLIMGRID_MIN (change),TMAX_NCLIMGRID_MEAN (change),TMAX_NCLIMGRID_MAX (change),TMAX_LIVNEH_MIN (change),TMAX_LIVNEH_MEAN (change),TMAX_LIVNEH_MAX (change),TAVG_NCLIMGRID_MIN (change),TAVG_NCLIMGRID_MEAN (change),TAVG_NCLIMGRID_MAX (change),...,PRDAYSABVNZ95TH_NCLIMGRID_MAX (change),PRDAYSABVNZ95TH_LIVNEH_MIN (change),PRDAYSABVNZ95TH_LIVNEH_MEAN (change),PRDAYSABVNZ95TH_LIVNEH_MAX (change),PRDAYSABVNZ99TH_NCLIMGRID_MIN (change),PRDAYSABVNZ99TH_NCLIMGRID_MEAN (change),PRDAYSABVNZ99TH_NCLIMGRID_MAX (change),PRDAYSABVNZ99TH_LIVNEH_MIN (change),PRDAYSABVNZ99TH_LIVNEH_MEAN (change),PRDAYSABVNZ99TH_LIVNEH_MAX (change)
0,20117,1.54364,1.71683,1.57245,1.21448,1.64418,1.83538,1.70422,1.66812,1.55216,...,-1,-3.0,-0.365079,1.0,0,1.730770,2,0.0,0.523810,1.0
1,20201,2.77005,2.27677,1.92838,2.60540,2.38334,2.19138,2.32867,2.05440,1.79876,...,0,1.0,-1.269842,-2.0,1,-0.207143,1,0.0,0.031746,-1.0
2,20203,2.22816,2.24628,2.26542,0.52685,0.56107,0.70411,1.29348,1.18545,1.10354,...,-1,1.0,-0.918367,-1.0,0,0.446281,0,0.0,-0.102041,0.0


In [75]:
# Calculate percentage difference between start and end years
variables_difference_percentage = ((gdf_end_change - gdf_start_change) / gdf_start_change) * 100

# Rename columns to reflect that they represent the percentage difference between the start and end years
variables_difference_percentage.rename(columns={gdf.columns[i]: f"{gdf.columns[i]} (% change)" for i in range(2, len(gdf.columns))}, inplace=True)

# Add index numbers to rows
variables_difference_percentage.reset_index(inplace=True)

# Show percentage difference of each county and each variable between the start and end years
variables_difference_percentage

,GEOID,TMAX_NCLIMGRID_MIN (% change),TMAX_NCLIMGRID_MEAN (% change),TMAX_NCLIMGRID_MAX (% change),TMAX_LIVNEH_MIN (% change),TMAX_LIVNEH_MEAN (% change),TMAX_LIVNEH_MAX (% change),TAVG_NCLIMGRID_MIN (% change),TAVG_NCLIMGRID_MEAN (% change),TAVG_NCLIMGRID_MAX (% change),...,PRDAYSABVNZ95TH_NCLIMGRID_MAX (% change),PRDAYSABVNZ95TH_LIVNEH_MIN (% change),PRDAYSABVNZ95TH_LIVNEH_MEAN (% change),PRDAYSABVNZ95TH_LIVNEH_MAX (% change),PRDAYSABVNZ99TH_NCLIMGRID_MIN (% change),PRDAYSABVNZ99TH_NCLIMGRID_MEAN (% change),PRDAYSABVNZ99TH_NCLIMGRID_MAX (% change),PRDAYSABVNZ99TH_LIVNEH_MIN (% change),PRDAYSABVNZ99TH_LIVNEH_MEAN (% change),PRDAYSABVNZ99TH_LIVNEH_MAX (% change)
0,20117,2.561817,2.797385,2.515316,1.995294,2.672305,2.941864,3.444990,3.318319,3.047122,...,-10.0,-42.857143,-3.911561,8.333333,NaN,478.723428,100.000000,NaN,39.285760,33.333333
1,20201,4.602177,3.712668,3.092390,4.306704,3.831309,3.456351,4.708925,4.089236,3.527299,...,0.0,33.333333,-13.536386,-15.384615,inf,-10.139868,33.333333,NaN,1.980196,-25.000000
2,20203,3.484224,3.481557,3.482593,0.805676,0.852378,1.061095,2.548428,2.313414,2.127547,...,-20.0,100.000000,-21.327008,-14.285714,NaN,1350.000227,0.000000,NaN,-35.714278,0.000000
